In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from datetime import datetime, timedelta
from geopy.distance import geodesic
import random
from faker import Faker
fake = Faker("es_ES")
Faker.seed(2002)

random.seed(2002)


df = pd.read_csv('usuarios.csv', encoding='utf-8', index_col=0)
ids = np.unique(df['id_usuario'])

df2 = pd.read_csv('geograficos_10min.csv')
coord_ini = df2['Coordenadas_inicio']
coord_fin = df2['Coordenadas_fin']
nom_ini = df2['Nombre_inicio']
nom_fin = df2['Nombre_fin']
tiempo = df2['Minutos']

In [2]:
# comenzamos a inventarnos datos

motivo = ['Estudios', 'Trabajo', 'Deporte', 'Ocio', 'Otro']
nombre_rutina_deporte = ['Futbol', 'Baloncesto', 'Yoga', 'Gym', 'Tenis', 'Padel', 'Baile']

disponibilidad = ['Coche', 'Pasajero', 'Coche y pasajero']
dias = ['L', 'M', 'X', 'J', 'V', 'S', 'D']
ant_trayecto = ['Bus', 'Metro', 'Tren', 'Coche', 'Coche', 'Coche', 'Coche']
horas_salida = ['7:00']

while horas_salida[-1] != '21:00':
    hora_ultima = datetime.strptime(horas_salida[-1], '%H:%M')
    hora_next = hora_ultima + timedelta(minutes=5)
    hora_next = hora_next.strftime('%H:%M')
    horas_salida.append(hora_next)
    


nombre_rutina = []
id_usuario = []
lugar_salida = []
lugar_llegada = []
nom_salida = []
nom_llegada = []
hora_salida = []
hora_llegada = []
motive = []
days = []
disp = []
viejo_trayecto = []
id_rutina = []



for i in range(50000):
    id_usuario.append(random.choice(ids))
    
    
    i = random.randint(0, 3499)
    lugar_salida.append(coord_ini[i][1:-1])
    lugar_llegada.append(coord_fin[i][1:-1])
    nom_salida.append(nom_ini[i])
    nom_llegada.append(nom_fin[i])
    tiempo_min = tiempo[i]
    
    hora_sal = random.choice(horas_salida)
    hora_salida.append(hora_sal)
    hora_inicial_dt = datetime.strptime(hora_sal, '%H:%M')
    hora_final_dt = hora_inicial_dt + timedelta(minutes=tiempo_min)
    hora_final = hora_final_dt.strftime('%H:%M')
    hora_llegada.append(hora_final)

    
    m = random.choice(motivo)
    motive.append(m)
    
    if m == 'Estudios':
        d = random.choice(dias[:5])
        nom_rut = f'Estudios_universidad_{d.lower()}' # solo cogemos a que estudien en universidades
    elif m == 'Otro':
        d = random.choice(dias)
        nom_rut = f'[Indefinido]_{d.lower()}'
    elif m == 'Trabajo':
        d = random.choice(dias[:5])
        nom_rut = f'Trabajo_{d.lower()}'
    elif m == 'Ocio':
        d = random.choice(dias)
        nom_rut = f'Ocio_{d.lower()}'
    else:                                   # es deporte
        d = random.choice(dias)
        deport = random.choice(nombre_rutina_deporte)
        nom_rut = f'Deporte_{deport.lower()}_{d.lower()}'
    
    days.append(d)
    nombre_rutina.append(nom_rut)
    disp.append(random.choice(disponibilidad))
    viejo_trayecto.append(random.choice(ant_trayecto))

In [3]:
d = {'nombre_rutina':nombre_rutina, 'id_usuario': id_usuario, 'lugar_origen':lugar_salida, 'lugar_destino':lugar_llegada,
     'nombre_origen': nom_salida, 'nombre_destino': nom_llegada,
     'hora_salida':hora_salida, 'hora_llegada':hora_llegada, 'motivo':motive, 'dia':days, 'disponibilidad_coche':disp,
     'transporte':viejo_trayecto}

df = pd.DataFrame(d)
df.head()

,nombre_rutina,id_usuario,lugar_origen,lugar_destino,nombre_origen,nombre_destino,hora_salida,hora_llegada,motivo,dia,disponibilidad_coche,transporte
0,[Indefinido]_l,111831,"39.50122800718866, -0.41927966292614793","39.44984907483212, -0.3923317810900036","Carrer d'Elies Tormo, Pobles de l'Oest, València","Avinguda del Primer de Maig, Jesús, València",13:50,14:12,Otro,L,Pasajero,Tren
1,Estudios_universidad_x,982450,"39.47448582694143, -0.36973136061629475","39.52827350270283, -0.3752537306405829","Plaça de Tetuan, Ciutat Vella, València",[Desconocido],7:00,07:24,Estudios,X,Coche y pasajero,Tren
2,Deporte_futbol_x,220802,"39.487288123748414, -0.3832181679770428","39.46320872584864, -0.378861657627463","Passeig de l'Ullal, la Saïdia, València","Gran Via de Ramón y Cajal, Extramurs, València",14:15,14:25,Deporte,X,Pasajero,Bus
3,Deporte_tenis_v,362571,"39.464666255889306, -0.378869876395067","39.48122685723318, -0.357178565188734","Carrer de Bailén, Extramurs, València","Carrer de Vicent Barrera Cambra, Benimaclet, V...",15:25,15:35,Deporte,V,Coche y pasajero,Coche
4,Ocio_v,694362,"39.41972386577771, -0.3892850451039514","39.45630189278762, -0.3795688513211948","Calle de San Pascual, Alfafar","Avinguda de Peris i Valero, l'Eixample, València",07:50,08:09,Ocio,V,Coche,Tren


In [4]:
# RUTINAS DUPLICADAS

dias = ['L', 'M', 'X', 'J', 'V']
filas_duplicadas = []
for dia in dias:
    for motivo in ['Estudios', 'Trabajo']:
        filas_dia_motivo = df[(df['dia'] == dia) & (df['motivo'] == motivo)].copy()
        if not filas_dia_motivo.empty:
            siguiente_dia = dias[(dias.index(dia) + 1) % len(dias)]
            filas_dia_motivo['dia'] = siguiente_dia
            filas_dia_motivo['nombre_rutina'] = motivo + '_' + siguiente_dia.lower()
            filas_duplicadas.append(filas_dia_motivo)

nuevo_df = pd.concat([df] + filas_duplicadas, ignore_index=True)

print(len(df), len(nuevo_df))
df = nuevo_df

50000 69979


In [5]:
# NUEVAS FILAS DE ESTUDIOS Y TRABAJO

filas_necesarias = 70000 - 10 - len(df) 

# Crear una lista de Motivos alternativos (Estudios o Trabajo)
motivos_alternativos = ['Estudios', 'Trabajo']

# Seleccionar las filas cuyo Motivo es 'Ocio'
filas_ocio = df[df['motivo'] == 'Ocio']

# Si hay más filas de 'Ocio' de las necesarias, toma solo las primeras
if len(filas_ocio) > filas_necesarias:
    filas_ocio = filas_ocio.iloc[:filas_necesarias]

# Cambiar el Motivo a 'Estudios' o 'Trabajo' de manera aleatoria
filas_ocio['motivo'] = np.random.choice(motivos_alternativos, size=len(filas_ocio))

# Cambiar el Día a un día de semana si es Sábado o Domingo
dias_semana = ['L', 'M', 'X', 'J', 'V']
filas_ocio['dia'] = np.where(filas_ocio['dia'].isin(['S', 'D']), np.random.choice(dias_semana, size=len(filas_ocio)), filas_ocio['dia'])
filas_ocio['nombre_rutina'] = filas_ocio['motivo'] + '_' + filas_ocio['dia'].str.lower()


# Agregar las filas modificadas al DataFrame original
df = pd.concat([df, filas_ocio], ignore_index=True)

In [6]:
df.tail()

,nombre_rutina,id_usuario,lugar_origen,lugar_destino,nombre_origen,nombre_destino,hora_salida,hora_llegada,motivo,dia,disponibilidad_coche,transporte
69985,Trabajo_v,126015,"39.47939649928545, -0.3771206771992797","39.499527474914395, -0.42242998224472217","Plaça de l'Autor, Ciutat Vella, València","Carrer de l'Enginyer Aubán, Pobles de l'Oest, ...",15:15,15:34,Trabajo,V,Coche y pasajero,Coche
69986,Estudios_v,359248,"39.50971200414956, -0.44623920702002756","39.51481879478544, -0.41535239935653245","Alborgí, Paterna","Calle de Arturo Cervellera, Burjassot",14:40,14:53,Estudios,V,Coche,Coche
69987,Estudios_v,777169,"39.50293637495475, -0.35025506429004816","39.449733140747554, -0.3969585182627564","Calle del Decano Sanfeliu, Alboraia / Alboraya","Avinguda de les Tres Creus, Jesús, València",07:30,08:08,Estudios,V,Pasajero,Coche
69988,Estudios_l,275563,"39.47883390886615, -0.3720843014131642","39.44834644250786, -0.33141219772190644","Circuit 5K Jardí del Túria, la Saïdia, València","Carrer de la Mare de Déu dels Reis, Poblats Ma...",08:45,09:04,Estudios,L,Pasajero,Coche
69989,Estudios_l,707822,"39.52052760344113, -0.387615329474106","39.48387276527541, -0.39536020180169285","Camí de Benifaraig a Montcada, Pobles del Nord...","Avinguda del Metge Vicente Torrent, Campanar, ...",08:10,08:26,Estudios,L,Pasajero,Coche


# DATOS DE PRUEBA (TODOS DEBERÍAN SER MATCHING DE prueba_1)

nueva_fila = {'id_trayecto': 'R00000', 'nombre_rutina':'prueba_1', 'id_usuario':'913690', 
              'lugar_origen':["39.47679696739808, -0.3462473060000242"], 
              'lugar_destino':["39.405576, -0.406062"], 'hora_salida':'7:30', 'hora_llegada':'8:00', 'motivo':'Estudios',
              'dia':'L', 'disponibilidad_coche':'coche y pasajero', 'transporte':'Coche'}

# primera prueba
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con un lugar de salida cercano
nueva_fila['Id_usuario'] = '156939'
nueva_fila['Nombre_rutina'] = 'prueba_2'
nueva_fila['Lugar_salida'] = ["39.478350, -0.347192"]
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con un lugar de llegada cercano
nueva_fila['Nombre_rutina'] = 'prueba_3'
nueva_fila['Lugar_llegada'] = ["39.404948, -0.406213"]
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con un motivo distinto
nueva_fila['Nombre_rutina'] = 'prueba_4'
nueva_fila['Motivo'] = "Trabajo"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una hora de salida distinta
nueva_fila['Nombre_rutina'] = 'prueba_5'
nueva_fila['Hora_salida'] = "7:20"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una hora de salida distinta
nueva_fila['Nombre_rutina'] = 'prueba_6'
nueva_fila['Hora_salida'] = "7:25"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una hora de llegada distinta
nueva_fila['Nombre_rutina'] = 'prueba_7'
nueva_fila['Hora_llegada'] = "7:50"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una hora de llegada distinta
nueva_fila['Nombre_rutina'] = 'prueba_8'
nueva_fila['Hora_llegada'] = "7:55"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una disponibilidad distinta
nueva_fila['Nombre_rutina'] = 'prueba_9'
nueva_fila['Disponibilidad_coche'] = "Coche"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)

# mismo pero con una disponibilidad distinta
nueva_fila['Nombre_rutina'] = 'prueba_10'
nueva_fila['Disponibilidad_coche'] = "Pasajero"
df = pd.concat([df, pd.DataFrame(nueva_fila)], ignore_index=True)


In [7]:
from faker import Faker
fake = Faker("es_ES")
Faker.seed(123)

In [8]:
df['id_trayecto'] = "T" + df.apply(lambda _: str(fake.unique.random_int(min=11111, max=99999)), axis=1)
df

,nombre_rutina,id_usuario,lugar_origen,lugar_destino,nombre_origen,nombre_destino,hora_salida,hora_llegada,motivo,dia,disponibilidad_coche,transporte,id_trayecto
0,[Indefinido]_l,111831,"39.50122800718866, -0.41927966292614793","39.44984907483212, -0.3923317810900036","Carrer d'Elies Tormo, Pobles de l'Oest, València","Avinguda del Primer de Maig, Jesús, València",13:50,14:12,Otro,L,Pasajero,Tren,T17974
1,Estudios_universidad_x,982450,"39.47448582694143, -0.36973136061629475","39.52827350270283, -0.3752537306405829","Plaça de Tetuan, Ciutat Vella, València",[Desconocido],7:00,07:24,Estudios,X,Coche y pasajero,Tren,T46195
2,Deporte_futbol_x,220802,"39.487288123748414, -0.3832181679770428","39.46320872584864, -0.378861657627463","Passeig de l'Ullal, la Saïdia, València","Gran Via de Ramón y Cajal, Extramurs, València",14:15,14:25,Deporte,X,Pasajero,Bus,T22538
3,Deporte_tenis_v,362571,"39.464666255889306, -0.378869876395067","39.48122685723318, -0.357178565188734","Carrer de Bailén, Extramurs, València","Carrer de Vicent Barrera Cambra, Benimaclet, V...",15:25,15:35,Deporte,V,Coche y pasajero,Coche,T64488
4,Ocio_v,694362,"39.41972386577771, -0.3892850451039514","39.45630189278762, -0.3795688513211948","Calle de San Pascual, Alfafar","Avinguda de Peris i Valero, l'Eixample, València",07:50,08:09,Ocio,V,Coche,Tren,T46048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69985,Trabajo_v,126015,"39.47939649928545, -0.3771206771992797","39.499527474914395, -0.42242998224472217","Plaça de l'Autor, Ciutat Vella, València","Carrer de l'Enginyer Aubán, Pobles de l'Oest, ...",15:15,15:34,Trabajo,V,Coche y pasajero,Coche,T39019
69986,Estudios_v,359248,"39.50971200414956, -0.44623920702002756","39.51481879478544, -0.41535239935653245","Alborgí, Paterna","Calle de Arturo Cervellera, Burjassot",14:40,14:53,Estudios,V,Coche,Coche,T74277
69987,Estudios_v,777169,"39.50293637495475, -0.35025506429004816","39.449733140747554, -0.3969585182627564","Calle del Decano Sanfeliu, Alboraia / Alboraya","Avinguda de les Tres Creus, Jesús, València",07:30,08:08,Estudios,V,Pasajero,Coche,T48891
69988,Estudios_l,275563,"39.47883390886615, -0.3720843014131642","39.44834644250786, -0.33141219772190644","Circuit 5K Jardí del Túria, la Saïdia, València","Carrer de la Mare de Déu dels Reis, Poblats Ma...",08:45,09:04,Estudios,L,Pasajero,Coche,T74545


In [9]:
#control
df = df.drop_duplicates()
df.to_csv("trayectos.csv",index_label=None)

## Pasamos de trayectos a rutinas

In [10]:
df

,nombre_rutina,id_usuario,lugar_origen,lugar_destino,nombre_origen,nombre_destino,hora_salida,hora_llegada,motivo,dia,disponibilidad_coche,transporte,id_trayecto
0,[Indefinido]_l,111831,"39.50122800718866, -0.41927966292614793","39.44984907483212, -0.3923317810900036","Carrer d'Elies Tormo, Pobles de l'Oest, València","Avinguda del Primer de Maig, Jesús, València",13:50,14:12,Otro,L,Pasajero,Tren,T17974
1,Estudios_universidad_x,982450,"39.47448582694143, -0.36973136061629475","39.52827350270283, -0.3752537306405829","Plaça de Tetuan, Ciutat Vella, València",[Desconocido],7:00,07:24,Estudios,X,Coche y pasajero,Tren,T46195
2,Deporte_futbol_x,220802,"39.487288123748414, -0.3832181679770428","39.46320872584864, -0.378861657627463","Passeig de l'Ullal, la Saïdia, València","Gran Via de Ramón y Cajal, Extramurs, València",14:15,14:25,Deporte,X,Pasajero,Bus,T22538
3,Deporte_tenis_v,362571,"39.464666255889306, -0.378869876395067","39.48122685723318, -0.357178565188734","Carrer de Bailén, Extramurs, València","Carrer de Vicent Barrera Cambra, Benimaclet, V...",15:25,15:35,Deporte,V,Coche y pasajero,Coche,T64488
4,Ocio_v,694362,"39.41972386577771, -0.3892850451039514","39.45630189278762, -0.3795688513211948","Calle de San Pascual, Alfafar","Avinguda de Peris i Valero, l'Eixample, València",07:50,08:09,Ocio,V,Coche,Tren,T46048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69985,Trabajo_v,126015,"39.47939649928545, -0.3771206771992797","39.499527474914395, -0.42242998224472217","Plaça de l'Autor, Ciutat Vella, València","Carrer de l'Enginyer Aubán, Pobles de l'Oest, ...",15:15,15:34,Trabajo,V,Coche y pasajero,Coche,T39019
69986,Estudios_v,359248,"39.50971200414956, -0.44623920702002756","39.51481879478544, -0.41535239935653245","Alborgí, Paterna","Calle de Arturo Cervellera, Burjassot",14:40,14:53,Estudios,V,Coche,Coche,T74277
69987,Estudios_v,777169,"39.50293637495475, -0.35025506429004816","39.449733140747554, -0.3969585182627564","Calle del Decano Sanfeliu, Alboraia / Alboraya","Avinguda de les Tres Creus, Jesús, València",07:30,08:08,Estudios,V,Pasajero,Coche,T48891
69988,Estudios_l,275563,"39.47883390886615, -0.3720843014131642","39.44834644250786, -0.33141219772190644","Circuit 5K Jardí del Túria, la Saïdia, València","Carrer de la Mare de Déu dels Reis, Poblats Ma...",08:45,09:04,Estudios,L,Pasajero,Coche,T74545


In [11]:
# Identificar filas duplicadas basadas en las columnas especificadas
duplicados = df.duplicated(subset=['id_usuario', 'lugar_origen', 'lugar_destino', 'hora_salida', 'hora_llegada', 'motivo'], keep=False)

# Filtrar el DataFrame para mostrar solo las filas duplicadas
filas_duplicadas = df[duplicados]

# Mostrar las filas duplicadas
print(filas_duplicadas)

                nombre_rutina  id_usuario  \
1      Estudios_universidad_x      982450   
8                   Trabajo_m      233733   
13                  Trabajo_l      244755   
15     Estudios_universidad_x      641384   
19                  Trabajo_x      783361   
...                       ...         ...   
69974               Trabajo_l      816843   
69975               Trabajo_l      258210   
69976               Trabajo_l      773111   
69977               Trabajo_l      275788   
69978               Trabajo_l      415541   

                                   lugar_origen  \
1       39.47448582694143, -0.36973136061629475   
8       39.46265595168075, -0.46172547634770683   
13       39.47679696739808, -0.3462473060000242   
15      39.459153697192484, -0.3827539497388502   
19        39.50836721057233, -0.441530330897987   
...                                         ...   
69974  39.492830843796014, -0.39243040931334794   
69975  39.513050653598384, -0.41248790485508674   


In [15]:
import pandas as pd
from faker import Faker
fake = Faker()

# Función para determinar el valor de 'dias' según las condiciones dadas
def determinar_dias(grupo):
    dias = grupo['dia'].tolist()
    dias_unicos = sorted(set(dias), key=lambda x: ['L', 'M', 'X', 'J', 'V', 'S', 'D'].index(x))
    if len(dias_unicos) == 1:
        return dias_unicos[0]
    elif dias_unicos == ['L', 'X']:
        return 'L, X'
    elif dias_unicos == ['S', 'D']:
        return 'S, D'
    else:
        return ', '.join(dias_unicos)

# Agrupar y aplicar la función para obtener el DataFrame de rutinas
rutinas = df.groupby(['id_usuario', 'lugar_origen', 'lugar_destino', 'hora_salida', 'hora_llegada', 'motivo', 'disponibilidad_coche']).apply(determinar_dias).reset_index()
rutinas.columns = ['id_usuario', 'lugar_origen', 'lugar_destino', 'hora_salida', 'hora_llegada', 'motivo', 'disponibilidad_coche', 'dias']

# Asignar nombres de origen y destino
nombre_origen = df.groupby('lugar_origen')['nombre_origen'].first().reset_index()
nombre_destino = df.groupby('lugar_destino')['nombre_destino'].first().reset_index()
rutinas = pd.merge(rutinas, nombre_origen, on='lugar_origen')
rutinas = pd.merge(rutinas, nombre_destino, on='lugar_destino')
rutinas['id_rutina'] = 'R' + df.apply(lambda _: str(fake.unique.random_int(min=11111, max=99999)), axis=1)
rutinas

,id_usuario,lugar_origen,lugar_destino,hora_salida,hora_llegada,motivo,disponibilidad_coche,dias,nombre_origen,nombre_destino,id_rutina
0,111182,"39.42089318618263, -0.3893774670138119","39.48938391621882, -0.3797248104873931",18:15,18:45,Ocio,Pasajero,L,"Calle Francisco Baixauli, Alfafar","Carrer de Just Ramírez (arquitecte), la Saïdia...",R14921
1,142337,"39.42089318618263, -0.3893774670138119","39.48938391621882, -0.3797248104873931",08:50,09:20,Trabajo,Coche,"X, J","Calle Francisco Baixauli, Alfafar","Carrer de Just Ramírez (arquitecte), la Saïdia...",R17314
2,286417,"39.42089318618263, -0.3893774670138119","39.48938391621882, -0.3797248104873931",15:00,15:30,Deporte,Pasajero,S,"Calle Francisco Baixauli, Alfafar","Carrer de Just Ramírez (arquitecte), la Saïdia...",R14698
3,322800,"39.42089318618263, -0.3893774670138119","39.48938391621882, -0.3797248104873931",14:50,15:20,Estudios,Coche,"M, X","Calle Francisco Baixauli, Alfafar","Carrer de Just Ramírez (arquitecte), la Saïdia...",R90100
4,331559,"39.42089318618263, -0.3893774670138119","39.48938391621882, -0.3797248104873931",17:15,17:45,Otro,Coche y pasajero,L,"Calle Francisco Baixauli, Alfafar","Carrer de Just Ramírez (arquitecte), la Saïdia...",R61020
...,...,...,...,...,...,...,...,...,...,...,...
50006,870520,"39.46726737457161, -0.3796746918789417","39.47843356553863, -0.33985470468029555",13:15,13:30,Estudios,Coche y pasajero,"M, X","Carrer de Sant Vicent Màrtir, Extramurs, València","Avinguda dels Tarongers, Algirós, València",R26488
50007,943459,"39.46726737457161, -0.3796746918789417","39.47843356553863, -0.33985470468029555",12:40,12:55,Ocio,Coche y pasajero,V,"Carrer de Sant Vicent Màrtir, Extramurs, València","Avinguda dels Tarongers, Algirós, València",R96148
50008,957832,"39.46726737457161, -0.3796746918789417","39.47843356553863, -0.33985470468029555",15:10,15:25,Ocio,Pasajero,S,"Carrer de Sant Vicent Màrtir, Extramurs, València","Avinguda dels Tarongers, Algirós, València",R25933
50009,961151,"39.46726737457161, -0.3796746918789417","39.47843356553863, -0.33985470468029555",12:40,12:55,Otro,Coche,J,"Carrer de Sant Vicent Màrtir, Extramurs, València","Avinguda dels Tarongers, Algirós, València",R47538


In [16]:
df[df.id_usuario==322800]

,nombre_rutina,id_usuario,lugar_origen,lugar_destino,nombre_origen,nombre_destino,hora_salida,hora_llegada,motivo,dia,disponibilidad_coche,transporte,id_trayecto
19320,Deporte_padel_j,322800,"39.50016669203209, -0.3658832469963033","39.45092301354079, -0.3345534383926463","Avinguda dels Germans Machado, Rascanya, València","Carrer del Coll de Rates, Poblats Marítims, Va...",12:50,13:15,Deporte,J,Coche y pasajero,Coche,T36568
19371,Ocio_m,322800,"39.45411371914026, -0.36691364087868555","39.46716765449121, -0.3375938839611933","Carrer de l'Actriu Pepita Serrador, Quatre Car...","Carrer de José María Haro (Magistrat), Algirós...",16:55,17:06,Ocio,M,Pasajero,Tren,T63619
33121,[Indefinido]_x,322800,"39.520865314276385, -0.3781542128514641","39.467106610069656, -0.3436496006810047","Camí calvari, Pobles del Nord, València","Carrer de Jeroni de Montsoriu, Camins al Grau,...",18:35,19:00,Otro,X,Pasajero,Coche,T53871
35058,Estudios_universidad_m,322800,"39.42089318618263, -0.3893774670138119","39.48938391621882, -0.3797248104873931","Calle Francisco Baixauli, Alfafar","Carrer de Just Ramírez (arquitecte), la Saïdia...",14:50,15:20,Estudios,M,Coche,Coche,T69630
43103,Estudios_universidad_v,322800,"39.495502021697426, -0.3977213754062677","39.52330452812123, -0.39287393703690743","Carrer de la Florista, Benicalap, València","Camí dels Avenars, Pobles del Nord, València",11:55,12:11,Estudios,V,Coche,Coche,T94967
55433,Estudios_x,322800,"39.42089318618263, -0.3893774670138119","39.48938391621882, -0.3797248104873931","Calle Francisco Baixauli, Alfafar","Carrer de Just Ramírez (arquitecte), la Saïdia...",14:50,15:20,Estudios,X,Coche,Coche,T69618
67692,Estudios_l,322800,"39.495502021697426, -0.3977213754062677","39.52330452812123, -0.39287393703690743","Carrer de la Florista, Benicalap, València","Camí dels Avenars, Pobles del Nord, València",11:55,12:11,Estudios,L,Coche,Coche,T69258


In [17]:
rutinas.to_csv("rutinas.csv",index_label=None, encoding="utf-8")